# Trinocular Machine Learning Model for Holography
### Grace E. Chesmore and Jeff McMahon - McMahonCosmologyLab
Here we build a machine learning model for fitting three holography measurements, yielding the LAT mirror adjuster offsets. This ML model takes in three holography measurements ("trinocular") in the form of far-field beams, then converts to aperture fields. The aperture fields and known adjuster offsets are then used as training sets for the ML model (1000 training sets in total).

Prior to using the aperture fields as training sets, the aberrations are subtracted, yielding the pathlength differences due only to the adjuster offsets. Three aberration patterns are computed below. 

In [ ]:
import numpy as np
path_to_holosim_package = "/home/chesmore/Desktop/Code/holosim_paper/package/holosim-ml"
sys.path.append(path_to_holosim_package)
import tele_geo as tg
import ap_field as af
import ap_fitting as afit
import optics_analyze as oa

# Plotting settings
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
font_manager.fontManager.addfont(
    "/home/chesmore/.local/share/fonts/times-new-roman.ttf"
)
matplotlib.rcParams["font.family"] = "Times New Roman"
matplotlib.rcParams["font.size"] = 28
plt.rcParams["image.cmap"] = "magma"
plt.style.use("ggplot")
plt.rcParams["axes.unicode_minus"] = False
%matplotlib inline

# Machine Learning packages
import sklearn
from sklearn.linear_model import LinearRegression
import pickle

save = 0

rx_x = np.array([-519.62 * (3 / 2), 519.62 * (3 / 2), 0])
rx_z = np.array([-300 * (3 / 2), -300 * (3 / 2), 600 * (3 / 2)])
el = np.array([oa.el_offset(rx_z[0]), oa.el_offset(rx_z[1]), oa.el_offset(rx_z[2])])
az = np.array([oa.az_offset(rx_x[0]), oa.az_offset(rx_x[1]), oa.az_offset(rx_x[2])])

shift_A = ["xy", oa.sh_x(rx_x[0]), oa.sh_z(rx_z[0])]
shift_B = ["xy", oa.sh_x(rx_x[1]), oa.sh_z(rx_z[1])]
shift_C = ["y", oa.sh_z(rx_z[2])]

n_adj_m1 = 5 * 77
n_adj_m2 = 5 * 69


In [ ]:
### Computing aberrations at three rx positions ###

tele_geo = tg.initialize_telescope_geometry()
th = np.linspace(-np.pi / 2 - 0.28, -np.pi / 2 + 0.28, tele_geo.N_scan)
ph = np.linspace(np.pi / 2 - 0.28, np.pi / 2 + 0.28, tele_geo.N_scan)

rx1 = np.array([rx_x[0], 209.09, rx_z[0]])
tele_geo = tg.tele_geo_init(rx1[0], rx1[1], rx1[2], el[0], az[0])
rxmirror_A = af.ray_mirror_pts(rx1, tele_geo, th, ph)
dat_A = afit.take_measurement(
    np.zeros(77 * 5), np.zeros(77 * 5), 0, tele_geo, rxmirror_A
)
dat_A = np.loadtxt(dat_A)
x_A, y_A, meas_A, ampl_A, geo = afit.analyze_holography(
    dat_A, tele_geo, 0, 1, 0, shift_A
)
meas_A = np.where(
    (abs(ampl_A) / np.max(abs(ampl_A))) >= 0.3, meas_A - np.mean(meas_A), 0
)

rx2 = np.array([rx_x[1], 209.09, rx_z[1]])
tele_geo = tg.tele_geo_init(rx2[0], rx2[1], rx2[2], el[1], az[1])
rxmirror_B = af.ray_mirror_pts(rx2, tele_geo, th, ph)
dat_B = afit.take_measurement(
    np.zeros(77 * 5), np.zeros(77 * 5), 0, tele_geo, rxmirror_B
)
dat_B = np.loadtxt(dat_B)
x_B, y_B, meas_B, ampl_B, geo = afit.analyze_holography(
    dat_B, tele_geo, 0, 1, 0, shift_B
)
meas_B = np.where(
    (abs(ampl_B) / np.max(abs(ampl_B))) >= 0.3, meas_B - np.mean(meas_B), 0
)

rx3 = np.array([rx_x[2], 209.09, rx_z[2]])
tele_geo = tg.tele_geo_init(rx3[0], rx3[1], rx3[2], el[2], az[2])
rxmirror_C = af.ray_mirror_pts(rx3, tele_geo, th, ph)
dat_C = afit.take_measurement(
    np.zeros(77 * 5), np.zeros(77 * 5), 0, tele_geo, rxmirror_C
)
dat_C = np.loadtxt(dat_C)
x_C, y_C, meas_C, ampl_C, geo = afit.analyze_holography(
    dat_C, tele_geo, 0, 1, 0, shift_C
)
meas_C = np.where(
    (abs(ampl_C) / np.max(abs(ampl_C))) >= 0.3, meas_C - np.mean(meas_C), 0
)

## Reading in the training sets

In [ ]:
adj_tot2 = []
phases2 = []

iters = 1000

for ii in range(0, iters):

    if np.mod(ii + 1, 20) == 0:
        print("Reading in measurement " + str(ii + 1) + "/" + str(iters))

    rx1 = np.array([rx_x[0], 209.09, rx_z[0]])
    tele_geo = tg.tele_geo_init(rx1[0], rx1[1], rx1[2], el[0], az[0])
    dat_A = np.loadtxt(
        "/data/chesmore/sim_out/trinocular/rx-519.620-300/rx_"
        + str(rx1)
        + "_"
        + str(ii + 1)
        + ".txt"
    )
    x_A, y_A, phase_A, ampl_A, geo = afit.analyze_holography(
        dat_A, tele_geo, 0, 1, 0, shift_A
    )
    rx2 = np.array([rx_x[1], 209.09, rx_z[1]])
    tele_geo = tg.tele_geo_init(rx2[0], rx2[1], rx2[2], el[1], az[1])
    dat_B = np.loadtxt(
        "/data/chesmore/sim_out/trinocular/rx519.620-300/rx_"
        + str(rx2)
        + "_"
        + str(ii + 1)
        + ".txt"
    )
    x_B, y_B, phase_B, ampl_B, geo = afit.analyze_holography(
        dat_B, tele_geo, 0, 1, 0, shift_B
    )
    rx3 = np.array([rx_x[2], 209.09, rx_z[2]])
    tele_geo = tg.tele_geo_init(rx3[0], rx3[1], rx3[2], el[2], az[2])
    dat_C = np.loadtxt(
        "/data/chesmore/sim_out/trinocular/rx00600/rx_"
        + str(rx3)
        + "_"
        + str(ii + 1)
        + ".txt"
    )
    x_C, y_C, phase_C, ampl_C, geo = afit.analyze_holography(
        dat_C, tele_geo, 0, 1, 0, shift_C
    )

    phase_B = np.where(
        (abs(ampl_B) / np.max(abs(ampl_B))) >= 0.3, phase_B - np.mean(phase_B), 0
    )
    phase_A = np.where(
        (abs(ampl_A) / np.max(abs(ampl_A))) >= 0.3, phase_A - np.mean(phase_A), 0
    )
    phase_C = np.where(
        (abs(ampl_C) / np.max(abs(ampl_C))) >= 0.3, phase_C - np.mean(phase_C), 0
    )
    phase_A -= meas_A
    phase_B -= meas_B
    phase_C -= meas_C

    adj_m1 = np.loadtxt(
        "/data/chesmore/sim_out/trinocular/rx00600/adj_offsets_m1_"
        + str(ii + 1)
        + ".txt"
    )[
        0:n_adj_m1
    ]  # mm

    adj_m2 = np.loadtxt(
        "/data/chesmore/sim_out/trinocular/rx00600/adj_offsets_m2_"
        + str(ii + 1)
        + ".txt"
    )[
        0:n_adj_m2
    ]  # mm

    phases2.append(np.concatenate((phase_A, phase_B, phase_C)))
    adj_tot2.append(np.concatenate((adj_m1, adj_m2)))

    if ii == 0:
        f, axes = plt.subplots(1, 3, figsize=(15, 3.5), sharey=True)
        plt.suptitle("Trinocular holography measurements", fontsize=30, x=0.44, y=1.15)
        colors = axes[0].scatter(
            x_A, y_A, c=1e6 * phase_A / tele_geo.k, vmin=-100, vmax=100
        )
        axes[0].axis("equal")
        axes[1].scatter(x_B, y_B, c=1e6 * phase_B / tele_geo.k, vmin=-100, vmax=100)
        axes[2].scatter(x_C, y_C, c=1e6 * phase_C / tele_geo.k, vmin=-100, vmax=100)
        axes[1].axis("equal")
        axes[2].axis("equal")
        axes[0].set_xlabel("x [m]")
        axes[0].set_ylabel("y [m]")
        axes[1].set_xlabel("x [m]")
        axes[2].set_xlabel("x [m]")
        f.colorbar(colors, ax=axes, label=r"$\mu m$")
        plt.show()

## Training and saving the ML model

In [ ]:
######## Binocular Model ################
n_samples = np.shape(phases2)[0]
n_features = np.shape(phases2)[1]

# Define training datasets
X = phases2[0 : int(iters - 1)]
y_output = adj_tot2[0 : int(iters - 1)]

# Train the linear regression ML model
model2 = LinearRegression()  # Define ML algorithm
model2.fit(X, y_output)  # Train model with training datasets

# Introduce instances where we do not know the answer (holography measurement)
Xnew = np.reshape(phases2[int(iters - 1)], (1, n_features))
# Make a prediction
ynew = model2.predict(Xnew)

# Save the model
filename_bi2 = "model_trinocular.sav"
pickle.dump(model2, open(filename_bi2, "wb"))

In [ ]:
plt.plot(ynew[0])
plt.plot(adj_tot2[int(iters - 1)])
plt.xlim(0, 100)